## Import des librairies

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score, accuracy_score, auc, roc_curve, confusion_matrix
from xgboost import XGBClassifier

In [2]:
# Commenter l'une ou l'autre lignes suivant que l'on souhaite travailler sur le dataset enrichi ou non. 

data=pd.read_csv('enrichered-with-cluster.csv')
liste=data.index[data['RainTomorrow'].isnull()]
data.drop(liste , inplace=True)

data=data.iloc[:,2:]
data['Date']=pd.to_datetime(data['Date'])
data=data.iloc[:,:-2]
data=data.iloc[:,:-5].join(data.iloc[:,-1:])
data=data.drop(columns='Location')
data=data.rename(columns={"Labels": "Location"})
data.head(5)

,Date,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,Location
0,2008-12-01,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,...,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,0.0,0.0,Continental
1,2008-12-02,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,...,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,0.0,0.0,Continental
2,2008-12-03,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,...,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,0.0,0.0,Continental
3,2008-12-04,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,...,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,0.0,0.0,Continental
4,2008-12-05,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,...,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,0.0,0.0,Continental


In [3]:
#On convertit le type de la variable Date
data['Date']=data['Date'].astype('datetime64')

/tmp/ipykernel_240/2376692298.py:2: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  data['Date']=data['Date'].astype('datetime64')


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142193 entries, 0 to 145458
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Date           142193 non-null  datetime64[ns]
 1   MinTemp        141556 non-null  float64       
 2   MaxTemp        141871 non-null  float64       
 3   Rainfall       140787 non-null  float64       
 4   Evaporation    81350 non-null   float64       
 5   Sunshine       74377 non-null   float64       
 6   WindGustDir    132863 non-null  object        
 7   WindGustSpeed  132923 non-null  float64       
 8   WindDir9am     132180 non-null  object        
 9   WindDir3pm     138415 non-null  object        
 10  WindSpeed9am   140845 non-null  float64       
 11  WindSpeed3pm   139563 non-null  float64       
 12  Humidity9am    140419 non-null  float64       
 13  Humidity3pm    138583 non-null  float64       
 14  Pressure9am    128179 non-null  float64       
 15  

In [5]:
la = LabelEncoder()
data['RainToday'] = la.fit_transform(data['RainToday'])
data['RainTomorrow'] = la.fit_transform(data['RainTomorrow'])

# Data cleaning

In [6]:
data = pd.get_dummies(data)

In [7]:
data = data.dropna()

# Entraînement du modèle

In [8]:
cor = data.corr()
cor2 = data.corr()
cor = np.where(cor <0,cor*-1 , cor)
cor=pd.DataFrame(cor)
cor.columns=cor2.columns
cor.index =cor2.index 

/tmp/ipykernel_240/3685902465.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  cor = data.corr()
/tmp/ipykernel_240/3685902465.py:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  cor2 = data.corr()


In [9]:
cor

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW,Location_Continental,Location_Est,Location_Nord,Location_Sud
MinTemp,1.000000,0.749940,0.115248,0.509595,0.075757,0.133227,0.134638,0.150001,0.182607,0.072295,...,0.000336,0.066333,0.076941,0.076885,0.000066,0.083743,0.384197,0.170533,0.517574,0.151219
MaxTemp,0.749940,1.000000,0.065842,0.653135,0.464779,0.050740,0.012703,0.028445,0.500402,0.447072,...,0.026041,0.086555,0.070401,0.072535,0.003407,0.073867,0.166147,0.014522,0.375421,0.106787
Rainfall,0.115248,0.065842,1.000000,0.073939,0.243855,0.105338,0.050920,0.043308,0.258623,0.276014,...,0.020704,0.018552,0.003677,0.024548,0.015288,0.017705,0.096000,0.051810,0.082008,0.005162
Evaporation,0.509595,0.653135,0.073939,1.000000,0.370672,0.221870,0.208450,0.138085,0.557311,0.420953,...,0.019459,0.014295,0.004228,0.041805,0.037521,0.019612,0.043753,0.057012,0.114483,0.098181
Sunshine,0.075757,0.464779,0.243855,0.370672,1.000000,0.039792,0.001752,0.044286,0.502676,0.631821,...,0.023737,0.022974,0.020499,0.022935,0.013082,0.001499,0.016625,0.036703,0.063368,0.003983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WindDir3pm_WSW,0.083743,0.073867,0.017705,0.019612,0.001499,0.044109,0.007949,0.034098,0.012241,0.011973,...,0.067675,0.069517,0.074987,0.073800,0.066200,1.000000,0.044252,0.075534,0.094478,0.108730
Location_Continental,0.384197,0.166147,0.096000,0.043753,0.016625,0.055934,0.000538,0.099563,0.055559,0.287238,...,0.017404,0.044468,0.039625,0.020387,0.011903,0.044252,1.000000,0.489069,0.371069,0.434522
Location_Est,0.170533,0.014522,0.051810,0.057012,0.036703,0.041470,0.014326,0.016667,0.021296,0.199877,...,0.018576,0.038800,0.085286,0.036876,0.012512,0.075534,0.489069,1.000000,0.213253,0.249719
Location_Nord,0.517574,0.375421,0.082008,0.114483,0.063368,0.043064,0.004700,0.125971,0.017629,0.127013,...,0.028315,0.087838,0.101974,0.072653,0.020111,0.094478,0.371069,0.213253,1.000000,0.189468


In [10]:
x=data

In [11]:
x = data
y = data['RainTomorrow']
x.drop('RainTomorrow', inplace = True, axis = 1)
x.drop('qtyraintomorrow', inplace = True, axis = 1,errors='ignore')
#x.drop('qtyraintomorrow', inplace = True, axis = 1)
x.drop('Date', inplace = True, axis = 1)
x

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW,Location_Continental,Location_Est,Location_Nord,Location_Sud
6049,17.9,35.2,0.0,12.0,12.3,48.0,6.0,20.0,20.0,13.0,...,0,0,1,0,0,0,1,0,0,0
6050,18.4,28.9,0.0,14.8,13.0,37.0,19.0,19.0,30.0,8.0,...,1,0,0,0,0,0,1,0,0,0
6052,19.4,37.6,0.0,10.8,10.6,46.0,30.0,15.0,42.0,22.0,...,0,0,0,0,0,0,1,0,0,0
6053,21.9,38.4,0.0,11.4,12.2,31.0,6.0,6.0,37.0,22.0,...,0,0,0,0,0,1,1,0,0,0
6054,24.2,41.0,0.0,11.2,8.4,35.0,17.0,13.0,19.0,15.0,...,0,0,0,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142298,19.3,33.4,0.0,6.0,11.0,35.0,9.0,20.0,63.0,32.0,...,0,0,0,0,0,0,0,0,1,0
142299,21.2,32.6,0.0,7.6,8.6,37.0,13.0,11.0,56.0,28.0,...,0,0,0,0,0,0,0,0,1,0
142300,20.7,32.8,0.0,5.6,11.0,33.0,17.0,11.0,46.0,23.0,...,0,0,0,1,0,0,0,0,1,0
142301,19.5,31.8,0.0,6.2,10.6,26.0,9.0,17.0,62.0,58.0,...,0,0,0,0,0,0,0,0,1,0


In [12]:
y.value_counts()

0    45361
1    12729
Name: RainTomorrow, dtype: int64

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20)

## Interpolation des valeur manquantes

# Evaluation du modèle

In [14]:
rf = RandomForestClassifier(n_estimators = 200)
rf.fit(x_train, y_train)

RandomForestClassifier(n_estimators=200)

In [15]:
predicted = rf.predict(x_test)
print("La précision est de {} %".format(accuracy_score(predicted, y_test)*100))

La précision est de 86.4520571526941 %


In [16]:
pd.crosstab(predicted, y_test)

RainTomorrow,0,1
row_0,,
0,8707,1191
1,383,1337


In [17]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rf, x, y, cv=5)
print(scores)


[0.84704768 0.85083491 0.83422276 0.86090549 0.85522465]
